In [1]:
# Cellule 1 — Imports & Paths

import os
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

# Dossiers de travail
BASE_DIR = Path("/home/fabrice/Boneage")

CSV_PATH = BASE_DIR / "test_eliva25.csv"
IMG_DIR = BASE_DIR / "test_eliva25"
MASK_DIR = BASE_DIR / "pred_masks_eliva25"

OUTPUT_DIR = BASE_DIR / "test_eliva25_segmented"
OUTPUT_DIR.mkdir(exist_ok=True)

print("CSV :", CSV_PATH)
print("Images :", IMG_DIR)
print("Masques :", MASK_DIR)
print("Output :", OUTPUT_DIR)


CSV : /home/fabrice/Boneage/test_eliva25.csv
Images : /home/fabrice/Boneage/test_eliva25
Masques : /home/fabrice/Boneage/pred_masks_eliva25
Output : /home/fabrice/Boneage/test_eliva25_segmented


In [2]:
# Cellule 2 — Charger le CSV

df = pd.read_csv(CSV_PATH)

print("Nombre d'images dans le CSV :", len(df))
df.head()


Nombre d'images dans le CSV : 33


,id,male
0,1.png,False
1,2.png,False
2,3.png,True
3,4.png,False
4,5.png,False


In [6]:
# Cellule 3 — Fonctions utilitaires

def load_image(path):
    """Charge une image en niveaux de gris : taille native (~887x1090)."""
    return cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

def load_mask(path):
    """Charge un masque 224x224 et le convertit en 0/1."""
    mask = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        return None
    
    # binaire → 0/1
    _, mask_bin = cv2.threshold(mask, 1, 1, cv2.THRESH_BINARY)
    return mask_bin  # shape (224,224)

def resize_mask_to_image(mask, image_shape):
    """
    Resize du masque 224x224 vers la taille native de l'image (~887x1090).
    image_shape = (H, W)
    """
    h, w = image_shape
    mask_resized = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)
    return mask_resized


In [7]:
# Cellule 4 — Boucle de segmentation complète 

missing_masks = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    img_id = str(row["id"])
    filename = f"{img_id}"

    img_path = IMG_DIR / filename
    mask_path = MASK_DIR / filename

    # Charger image
    image = load_image(img_path)
    if image is None:
        print(f" Image introuvable : {filename}")
        continue

    # Charger masque 224x224
    mask224 = load_mask(mask_path)
    if mask224 is None:
        missing_masks.append(filename)
        continue

    # Resize le masque vers la taille de l'image
    mask_resized = resize_mask_to_image(mask224, image.shape)

    # Appliquer le masque
    masked_image = image * mask_resized

    # Sauvegarder
    save_path = OUTPUT_DIR / filename
    cv2.imwrite(str(save_path), masked_image)

print("Segmentation terminée.")


100%|███████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 34.19it/s]

Segmentation terminée.
